In [1]:
import telebot
import time
import pandas as pd
import numpy as np
import pathlib
from aiogram.utils.markdown import hbold, hlink
import difflib as dl
import random

In [27]:
from scripts.visualization import get_track_visualization
from scripts.characteristics_based import get_final_recs

TO DO:
1) lowercasing - done
2) multiple tracks vis
3) additional recs per track - permutation
4) hosting 
5) help - ваня 
6) estimation - остальные

In [17]:
def get_track_id(artist, song_name):
    id2track = pd.read_csv(str(pathlib.Path().parent.absolute()) + '/data/id2track.csv', index_col=['track_id'])
    track_id = id2track[(id2track.song_name == song_name) & (id2track.artist == artist)].index[0]
    return track_id
    

In [3]:
def load_track_data():
    return pd.read_csv(str(pathlib.Path().parent.absolute()) + '/data/id2track.csv', index_col=['track_id'])

In [29]:
bot = telebot.TeleBot('5218930942:AAGqwa9qRR6oEEhhfw6s8W38fbFq1gEecyc')

In [30]:

@bot.message_handler(commands = ['start'])
def start(message):
    hi_message = f'Привет, {message.from_user.first_name}!'
    bot.send_message(message.chat.id, hi_message + '\nНапиши боту "Я выбрал песню".')


#@bot.message_handler(commands= ['help'])
#def help(help_text):
#    bot.send_message(message.chat.id, 'Сейчас допишу')



@bot.message_handler(func = lambda message: message.text == 'Я выбрал песню')
def get_user_text(message):
        keyboard = telebot.types.InlineKeyboardMarkup()
        keyboard.row(telebot.types.InlineKeyboardButton('Визуализировать трек', callback_data = 'button_vis'))
        keyboard.row(telebot.types.InlineKeyboardButton('Получить рекомендацию', callback_data = 'button_rec'))
        bot.send_message(message.chat.id, 'Выбери, что ты хочешь увидеть.' + '\nКнопка "Визуализировать трек" покажет тебе график с музыкальными характеристиками. Например, танцевальностью.' + '\nКнопка "Получить рекомендацию" запустит процесс анализа трека и покажет тебе похoжие на него.', reply_markup = keyboard)



@bot.callback_query_handler(func = lambda call: call.data == 'button_vis')
def newbutton_vis(visualisation):
    msg = bot.send_message(visualisation.message.chat.id, 'Введи имя исполнителя!')
    bot.register_next_step_handler(msg, artist_name_step_vis)


def artist_name_step_vis(msg):
    id2track = load_track_data()
    artist = msg.text
    artist = dl.get_close_matches(artist, np.unique(id2track.artist.values).tolist(), n = 1)[0]
    msg = bot.send_message(msg.chat.id, f'Введи название трека у {artist}!')
    bot.register_next_step_handler(msg, visualization_step, artist)

def visualization_step(msg, artist):
    id2track = load_track_data()
    track_name = msg.text
    track_name = dl.get_close_matches(track_name, np.unique(id2track[id2track.artist == artist].song_name.values).tolist(), n = 1)[0]
    try:
        plot = get_track_visualization(artist, track_name)
        plot.write_image("images/fig1.pdf")#, width=1080, height=1080)
        bot.send_photo(msg.chat.id, photo = open('images/fig1.pdf', 'rb'))

        keyboard_back = telebot.types.InlineKeyboardMarkup()
        keyboard_back.row(telebot.types.InlineKeyboardButton('Вернуться к выбору опций', callback_data = 'button_back'))
        bot.send_message(msg.chat.id,  'Нажми на кнопку, если хочешь проверить еще один трек!', reply_markup = keyboard_back)
    except: #returns to initial choice
        keyboard_back = telebot.types.InlineKeyboardMarkup()
        keyboard_back.row(telebot.types.InlineKeyboardButton('Попробовать еще раз', callback_data = 'button_back'))
        bot.send_message(msg.chat.id,  'Проверь имя исполнителя и название трека!', reply_markup = keyboard_back)    





@bot.callback_query_handler(func = lambda call: call.data == 'button_rec')
def newbutton(query):
    msg = bot.send_message(query.message.chat.id, 'Введи имя исполнителя!')
    bot.register_next_step_handler(msg, artist_name_step)


def artist_name_step(msg):
    id2track = load_track_data()
    artist = msg.text
    artist = dl.get_close_matches(artist, np.unique(id2track.artist.values).tolist(), n = 1)[0]
    msg = bot.send_message(msg.chat.id, f'Введи название трека у {artist}!')
    bot.register_next_step_handler(msg, track_name_step, artist)

def track_name_step(msg, artist): 
    id2track = load_track_data()
    track_name = msg.text
    track_name = dl.get_close_matches(track_name, np.unique(id2track[id2track.artist == artist].song_name.values).tolist(), n = 1)[0]

    user_data = {}
    user_data['artist'] = artist
    user_data['song_name'] = track_name
    msg = bot.send_message(msg.chat.id, f'Сколько треков тебе порекомендовать?')
    bot.register_next_step_handler(msg, recommendation_step, user_data)

def recommendation_step(msg, user_data):
    user_data['n'] = msg.text
    try: 
        track_id = get_track_id(user_data['artist'], user_data['song_name'])
        recommendations = get_final_recs(track_id, int(user_data['n']))
        for key in recommendations.keys():
            bot.send_message(msg.chat.id, f"{hlink(recommendations[key], key)}", parse_mode='HTML')

        keyboard_back = telebot.types.InlineKeyboardMarkup()
        keyboard_back.row(telebot.types.InlineKeyboardButton('Вернуться к выбору опций', callback_data = 'button_back'))
        bot.send_message(msg.chat.id,  'Выбери, что делать дальше!', reply_markup = keyboard_back)
    

    except: #returns to initial choice
        keyboard_back = telebot.types.InlineKeyboardMarkup()
        keyboard_back.row(telebot.types.InlineKeyboardButton('Попробовать еще раз', callback_data = 'button_back'))
        bot.send_message(msg.chat.id,  'Проверь имя исполнителя и название трека!', reply_markup = keyboard_back)    



@bot.callback_query_handler(func = lambda call: call.data == 'button_back')
def get_user_text_back(message_back):
        keyboard = telebot.types.InlineKeyboardMarkup()
        keyboard.row(telebot.types.InlineKeyboardButton('Визуализировать трек', callback_data = 'button_vis'))
        keyboard.row(telebot.types.InlineKeyboardButton('Получить рекомендацию', callback_data = 'button_rec'))
        bot.send_message(message_back.message.chat.id, 'Выбери, что ты хочешь увидеть.' + '\nКнопка "Визуализировать трек" покажет тебе график с музыкальными характеристиками. Например, танцевальностью.' + '\nКнопка "Получить рекомендацию" запустит процесс анализа трека и покажет тебе похoжие на него.', reply_markup = keyboard)

#track list by artist
@bot.message_handler(func = lambda message: message.text == 'test')
def get_artist_name(message):
    msg = bot.send_message(message.chat.id, 'Выбери исполнителя!')
    bot.register_next_step_handler(msg, get_song_name)

def get_song_name(msg):
    id2track = load_track_data()
    tracks = id2track[id2track.artist == msg.text]['song_name'].values
    print(tracks)
    bot.send_message(msg.chat.id, tracks)
    





if __name__ == '__main__': # чтобы код выполнялся только при запуске в виде сценария, а не при импорте модуля
    try:
       bot.polling(none_stop=True) # запуск бота
    except Exception as e:
       print(e) # или import traceback; traceback.print_exc() для печати полной инфы
       time.sleep(15)

HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)


In [142]:
bot.stop_polling()